# Auto-Generated FastAPI Pipeline Notebook

This notebook fine-tunes CodeT5, generates FastAPI code, UT/FT, injects into a template, zips, and can push to GitHub.

In [ ]:
# Replace with your actual repo URL
!git clone https://github.com/ASHOKEKUMAR-S/codet5_fastapi_finetune.git

# Change directory into the repo
%cd codet5-fastapi-pipeline


## SECTION 1: Upload your .jsonl training data

In [ ]:
from google.colab import files
import json

uploaded = files.upload()
filename = list(uploaded.keys())[0]
with open(filename, 'r') as f:
    dataset = [json.loads(line) for line in f]

print("Sample record:", dataset[0])

## SECTION 2: Tokenize training data

In [ ]:
from transformers import AutoTokenizer
from datasets import Dataset

tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5-small")
max_source_length = 512
max_target_length = 512

def preprocess_data(examples):
    inputs = tokenizer(examples['inputs'], max_length=max_source_length, padding="max_length", truncation=True)
    targets = tokenizer(examples['targets'], max_length=max_target_length, padding="max_length", truncation=True)
    inputs["labels"] = targets["input_ids"]
    return inputs

hf_dataset = Dataset.from_list(dataset)
tokenized_dataset = hf_dataset.map(preprocess_data, batched=True)

## SECTION 3: Fine-tune CodeT5 model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, TrainingArguments, Trainer

model = AutoModelForSeq2SeqLM.from_pretrained("Salesforce/codet5-small")

training_args = TrainingArguments(
    output_dir="./codet5_finetuned_fastapi",
    per_device_train_batch_size=4,
    num_train_epochs=2,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

trainer.train()

## SECTION 4: Generate FastAPI code from a prompt

In [ ]:
def generate_code(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
    output = model.generate(**inputs, max_length=512)
    return tokenizer.decode(output[0], skip_special_tokens=True)

sample_prompt = """Create a FastAPI microservice to return all distinct environments from a devices table.

Table Design:
Table: devices
Columns:
- id: integer, primary key
- device_id: varchar(64)
- hostname: varchar(64)
- application: varchar(32)
- environment: varchar(16)
- os_type: varchar(16)
- os_version: varchar(16)
- tech_stack: varchar(64)
"""

generated_code = generate_code(sample_prompt)

with open("generated_fastapi_code.py", "w") as f:
    f.write(generated_code)

print("Code saved to generated_fastapi_code.py")

## SECTION 5: Split and inject into template folder

In [ ]:
import os

with open("generated_fastapi_code.py", "r") as f:
    code = f.read()

if '# <models.py>' in code and '# <routes.py>' in code:
    model_code = code.split('# <models.py>')[1].split('# <routes.py>')[0].strip()
    route_code = code.split('# <routes.py>')[1].strip()

    with open("app/models.py", "w") as f:
        f.write(model_code)

    with open("app/routes.py", "w") as f:
        f.write(route_code)

    print("Injected into app/models.py and app/routes.py")
else:
    print("⚠️ Expected code markers not found.")

## SECTION 6: Generate unit test (optional)

In [ ]:
test_prompt = f"""Generate pytest unit tests for the following FastAPI route:

```python
{route_code}
```
"""

test_code = generate_code(test_prompt)

with open("tests/test_unit.py", "w") as f:
    f.write(test_code)

print("✅ Unit test saved to tests/test_unit.py")

## SECTION 7: Generate functional test (optional)

In [ ]:
ft_prompt = f"""Generate functional tests for this FastAPI app using TestClient:

```python
{route_code}
```
"""

ft_code = generate_code(ft_prompt)

with open("tests/test_functional.py", "w") as f:
    f.write(ft_code)

print("✅ Functional test saved to tests/test_functional.py")

## SECTION 8: Zip final app for delivery

In [ ]:
import shutil

shutil.make_archive("final_fastapi_app", 'zip', ".")
print("🎉 Final app zipped: final_fastapi_app.zip")